In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from geopy import Nominatim


In [2]:
ratings = pd.read_csv('../data/chocolate_ratings.csv')
ratings.shape, ratings.columns

((2530, 10),
 Index(['REF', 'Company (Manufacturer)', 'Company Location', 'Review Date',
        'Country of Bean Origin', 'Specific Bean Origin or Bar Name',
        'Cocoa Percent', 'Ingredients', 'Most Memorable Characteristics',
        'Rating'],
       dtype='object'))

In [3]:
def findGeocode(place):  
    '''Finds the geocode of any location'''
    try:
        geolocator = Nominatim(user_agent="") ##user agent should be created
        return geolocator.geocode(place) 
    except Exception as e:
        print(e)
        return None


In [7]:
places = list(ratings['Company Location'].unique())

lat_long = []
# use geopy to extract coordinates from place names
for place in tqdm(places): 
        loc = findGeocode(place) 
        lat_long.append({'latitude': loc.latitude, 'longitude': loc.longitude})


100%|██████████| 67/67 [00:33<00:00,  2.01it/s]


In [8]:
lat_long[:2]

[{'latitude': 28.41769265, 'longitude': -81.58111071077724},
 {'latitude': 46.603354, 'longitude': 1.8883335}]

In [13]:
with open('country_locations.json', 'w') as outfile:
    json.dump(lat_long, outfile, )

In [15]:
makers = pd.read_csv('../data/chocolate_makers.csv')
makers.shape, makers.columns

((373, 5),
 Index(['COMPANY NAME', 'CITY', 'STATE/PROVINCE', 'OWNER/MAKER', 'COUNTRY'], dtype='object'))

In [17]:
places = list(makers['STATE/PROVINCE'].unique())

lat_long = []
# use geopy to extract coordinates from place names
for place in tqdm(places): 
        loc = findGeocode(place) 
        lat_long.append({'latitude': loc.latitude, 'longitude': loc.longitude})


100%|██████████| 59/59 [00:32<00:00,  1.82it/s]


In [18]:
with open('state_locations.json', 'w') as outfile:
    json.dump(lat_long, outfile, )

In [21]:
places = list(makers['CITY'].unique())

lat_long = []
# use geopy to extract coordinates from place names
for place in tqdm(places): 
        loc = findGeocode(place) 
        try:
                lat_long.append({'latitude': loc.latitude, 'longitude': loc.longitude})
        except:
                print(place)

 21%|██        | 55/266 [00:27<01:43,  2.03it/s]

Ascata


 41%|████      | 108/266 [00:53<01:24,  1.87it/s]

Anahola (Kaua'I)


 71%|███████   | 189/266 [01:35<00:40,  1.90it/s]

Swamanoa


 82%|████████▏ | 217/266 [01:49<00:24,  2.03it/s]

Chattonooga


100%|██████████| 266/266 [02:14<00:00,  1.97it/s]


In [22]:
with open('city_locations.json', 'w') as outfile:
    json.dump(lat_long, outfile, )